# NLP using Word Vectors

### Regular word ecodding (one-hot-encodding)

### What are word vectors

### How to use word vectors for comparing word similarity

### How to get a basic document vector

### Improve document vector through stop word removal